## Channel Details

In [21]:
import requests
import csv

# API details
API_KEY = "YOUR_API_KEY"
CHANNEL_ID = "UCSaJZiQye_U2ZQXo85bQFLw"
BASE_URL = "https://www.googleapis.com/youtube/v3/channels"
PART = "contentDetails,contentOwnerDetails,id,snippet,statistics,status,topicDetails"

# Function to get channel data
def get_channel_data(api_key, channel_id):
    params = {
        "key": api_key,
        "id": channel_id,
        "part": PART
    }
    response = requests.get(BASE_URL, params=params)
    response.raise_for_status()
    return response.json()

# Function to extract relevant data from the API response
def extract_channel_info(channel_data):
    channel_info = channel_data.get("items", [])[0]  # Assuming only one item in the response
    snippet = channel_info.get("snippet", {})
    statistics = channel_info.get("statistics", {})
    content_details = channel_info.get("contentDetails", {})
    content_owner_details = channel_info.get("contentOwnerDetails", {})
    status = channel_info.get("status", {})
    topic_details = channel_info.get("topicDetails", {})

    return {
        "channelId": channel_info.get("id", "N/A"),
        "title": snippet.get("title", "N/A"),
        "description": snippet.get("description", "N/A"),
        "publishedAt": snippet.get("publishedAt", "N/A"),
        "country": snippet.get("country", "N/A"),
        "viewCount": statistics.get("viewCount", "N/A"),
        "subscriberCount": statistics.get("subscriberCount", "N/A"),
        "videoCount": statistics.get("videoCount", "N/A"),
        "hiddenSubscriberCount": statistics.get("hiddenSubscriberCount", "N/A"),
        "uploadsPlaylistId": content_details.get("relatedPlaylists", {}).get("uploads", "N/A"),
        "favoritesPlaylistId": content_details.get("relatedPlaylists", {}).get("favorites", "N/A"),
        "watchLaterPlaylistId": content_details.get("relatedPlaylists", {}).get("watchLater", "N/A"),
        "contentOwnerDetails": content_owner_details.get("contentOwner", "N/A"),
        "status": status.get("privacyStatus", "N/A"),
        "topicDetails": topic_details.get("topicCategories", "N/A")
    }

# Function to export data to CSV
def export_to_csv(data, filename="youtube_channel_details.csv"):
    with open(filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        # Write headers
        writer.writerow([
            "Channel ID", "Title", "Description", "Published At", "Country", "View Count", 
            "Subscriber Count", "Video Count", "Hidden Subscriber Count", "Uploads Playlist ID", 
            "Favorites Playlist ID", "Watch Later Playlist ID", "Content Owner", "Privacy Status", "Topic Categories"
        ])
        # Write data rows
        writer.writerow([
            data["channelId"], data["title"], data["description"], data["publishedAt"], 
            data["country"], data["viewCount"], data["subscriberCount"], data["videoCount"], 
            data["hiddenSubscriberCount"], data["uploadsPlaylistId"], data["favoritesPlaylistId"], 
            data["watchLaterPlaylistId"], data["contentOwnerDetails"], data["status"], data["topicDetails"]
        ])

# Main function
if __name__ == "__main__":
    # Get channel data from API
    channel_data = get_channel_data(API_KEY, CHANNEL_ID)
    # Extract relevant data from the API response
    channel_info = extract_channel_info(channel_data)
    # Export the data to CSV
    export_to_csv(channel_info)
    print(f"Exported channel details to youtube_channel_details.csv.")


Exported channel details to youtube_channel_details.csv.


## Video Details

In [14]:
import requests
import csv

# API details
API_KEY = "YOUR_API_KEY"
CHANNEL_ID = "UCSaJZiQye_U2ZQXo85bQFLw"  # Channel ID for Edurise Point
BASE_URL_SEARCH = "https://www.googleapis.com/youtube/v3/search"
BASE_URL_VIDEOS = "https://www.googleapis.com/youtube/v3/videos"
PART_SNIPPET = "snippet"
PART_STATISTICS = "statistics"
PART_CONTENT = "contentDetails"
TYPE = "video"
MAX_RESULTS = 50

# Function to search videos from the specified channel
def search_videos(api_key, channel_id, max_results=50, page_token=None):
    params = {
        "key": api_key,
        "channelId": channel_id,
        "type": TYPE,
        "part": PART_SNIPPET,
        "maxResults": max_results,
    }
    if page_token:
        params["pageToken"] = page_token

    response = requests.get(BASE_URL_SEARCH, params=params)
    response.raise_for_status()
    return response.json()

# Function to get video statistics
def get_video_statistics(api_key, video_ids):
    params = {
        "key": api_key,
        "id": ",".join(video_ids),
        "part": PART_STATISTICS + "," + PART_SNIPPET + "," + PART_CONTENT,
    }

    response = requests.get(BASE_URL_VIDEOS, params=params)
    response.raise_for_status()
    return response.json()

# Gather all video data for the channel
def fetch_video_details(api_key, channel_id):
    all_video_details = []
    next_page_token = None

    while True:
        # Search for videos from the specified channel
        search_data = search_videos(api_key, channel_id, MAX_RESULTS, next_page_token)
        video_ids = [item["id"]["videoId"] for item in search_data["items"]]

        # Get video statistics
        video_stats = get_video_statistics(api_key, video_ids)
        for video in video_stats["items"]:
            snippet = video.get("snippet", {})
            statistics = video.get("statistics", {})
            content = video.get("contentDetails", {})

            details = {
                "videoId": video.get("id", "N/A"),
                "title": snippet.get("title", "N/A"),
                "description": snippet.get("description", "N/A"),
                "publishedAt": snippet.get("publishedAt", "N/A"),
                "channelTitle": snippet.get("channelTitle", "N/A"),
                "tags": snippet.get("tags", "N/A"),
                "viewCount": statistics.get("viewCount", "N/A"),
                "likeCount": statistics.get("likeCount", "N/A"),
                "commentCount": statistics.get("commentCount", "N/A"),
                "duration": content.get("duration", "N/A")
            }
            all_video_details.append(details)

        # Check if there is another page of results
        next_page_token = search_data.get("nextPageToken")
        if not next_page_token:
            break

    return all_video_details

# Export data to CSV
def export_to_csv(data, filename="youtube_video_details.csv"):
    with open(filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        # Write headers
        writer.writerow(["Video ID", "Title", "Description", "Published At", "Channel Title", "View Count", "Like Count", "Comment Count", "Video Duration", "Video Tags"])
        # Write data rows
        for item in data:
            writer.writerow([item["videoId"], item["title"], item["description"], item["publishedAt"], item["channelTitle"], item["viewCount"], item["likeCount"], item["commentCount"], item["duration"], item["tags"]])

# Main function
if __name__ == "__main__":
    video_details = fetch_video_details(API_KEY, CHANNEL_ID)
    print(f"Fetched details for {len(video_details)} videos.")
    export_to_csv(video_details)
    print("Exported video details to youtube_video_details.csv.")

Fetched details for 500 videos.
Exported video details to youtube_video_details.csv.


## Comments

In [2]:
import requests
import csv

# API details
API_KEY = "YOUR_API_KEY"
CHANNEL_ID = "UCSaJZiQye_U2ZQXo85bQFLw"  # Channel ID
BASE_URL_VIDEOS = "https://www.googleapis.com/youtube/v3/playlistItems"
BASE_URL_COMMENTS = "https://www.googleapis.com/youtube/v3/commentThreads"
BASE_URL_VIDEO_DETAILS = "https://www.googleapis.com/youtube/v3/videos"
PART = "snippet,replies"
MAX_RESULTS = 100  # Max comments per request

# Function to get the playlist name from the channel
def get_playlist_name_from_channel(api_key, channel_id):
    playlist_id = "UU" + channel_id[2:]  # This is the "Uploads" playlist ID
    params = {
        "key": api_key,
        "playlistId": playlist_id,
        "part": "snippet",
        "maxResults": 1,
    }
    response = requests.get(BASE_URL_VIDEOS, params=params)
    response.raise_for_status()
    data = response.json()
    if "items" in data:
        return data["items"][0]["snippet"]["playlistId"]
    return None

# Function to get video IDs from the playlist
def get_video_ids_from_playlist(api_key, playlist_id):
    video_ids = []
    next_page_token = None
    
    while True:
        params = {
            "key": api_key,
            "playlistId": playlist_id,
            "part": "snippet",
            "maxResults": MAX_RESULTS,
        }
        if next_page_token:
            params["pageToken"] = next_page_token
        
        response = requests.get(BASE_URL_VIDEOS, params=params)
        response.raise_for_status()
        data = response.json()
        
        # Collect video IDs from the playlist
        video_ids.extend([item["snippet"]["resourceId"]["videoId"] for item in data["items"]])
        
        # Get next page token if available
        next_page_token = data.get("nextPageToken")
        if not next_page_token:
            break  # No more pages, exit the loop
    
    return video_ids

# Function to get video details (title) from video ID
def get_video_details(api_key, video_id):
    params = {
        "key": api_key,
        "id": video_id,
        "part": "snippet",
    }
    response = requests.get(BASE_URL_VIDEO_DETAILS, params=params)
    response.raise_for_status()
    data = response.json()
    video_title = data["items"][0]["snippet"]["title"]
    return video_title

# Function to get comments for a video
def get_video_comments(api_key, video_id, page_token=None):
    params = {
        "key": api_key,
        "videoId": video_id,
        "part": PART,
        "maxResults": MAX_RESULTS,
    }
    if page_token:
        params["pageToken"] = page_token
    
    response = requests.get(BASE_URL_COMMENTS, params=params)
    response.raise_for_status()
    return response.json()

# Function to fetch all comments for a video
def fetch_all_comments_for_video(api_key, video_id, video_title, playlist_name):
    all_comments = []
    next_page_token = None
    
    while True:
        comments_data = get_video_comments(api_key, video_id, next_page_token)
        
        for item in comments_data["items"]:
            comment = item["snippet"]["topLevelComment"]["snippet"]
            comment_details = {
                "playlist_name": playlist_name,  # Store playlist name with the comment
                "video_title": video_title,      # Store video title with the comment
                "video_id": video_id,            # Store video ID with the comment
                "author": comment.get("authorDisplayName", "N/A"),
                "comment": comment.get("textDisplay", "N/A"),
                "publishedAt": comment.get("publishedAt", "N/A"),
                "likeCount": comment.get("likeCount", "N/A"),
                "authorChannelUrl": comment.get("authorChannelUrl", "N/A"),
            }
            all_comments.append(comment_details)
        
        next_page_token = comments_data.get("nextPageToken")
        if not next_page_token:
            break  # No more pages of comments
    
    return all_comments

# Function to export comments to CSV
def export_comments_to_csv(comments, filename="youtube_video_comments.csv"):
    with open(filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        # Write headers, including video_id and playlist_name
        writer.writerow(["Playlist Name", "Video Title", "Video ID", "Author", "Comment", "Published At", "Like Count", "Author Channel URL"])
        # Write comment data
        for comment in comments:
            writer.writerow([comment["playlist_name"], comment["video_title"], comment["video_id"], comment["author"], comment["comment"], comment["publishedAt"], comment["likeCount"], comment["authorChannelUrl"]])

# Main function
if __name__ == "__main__":
    # Step 1: Get the playlist name from the channel
    playlist_name = get_playlist_name_from_channel(API_KEY, CHANNEL_ID)
    if not playlist_name:
        print("Error: Playlist name not found.")
    else:
        print(f"Found playlist: {playlist_name}")

    # Step 2: Get all video IDs from the playlist
    video_ids = get_video_ids_from_playlist(API_KEY, playlist_name)
    print(f"Found {len(video_ids)} videos in the playlist.")
    
    # Step 3: Fetch all comments for each video
    all_comments = []
    for video_id in video_ids:
        print(f"Fetching comments for video {video_id}...")
        video_title = get_video_details(API_KEY, video_id)  # Get video title
        video_comments = fetch_all_comments_for_video(API_KEY, video_id, video_title, playlist_name)
        all_comments.extend(video_comments)
    
    # Step 4: Export all comments to CSV
    print(f"Fetched {len(all_comments)} comments in total.")
    export_comments_to_csv(all_comments)
    print("Exported comments to youtube_video_comments.csv.")

Found playlist: UUSaJZiQye_U2ZQXo85bQFLw
Found 105 videos in the playlist.
Fetching comments for video EZNeU4G1yNI...
Fetching comments for video 6ABBZ8DkPXU...
Fetching comments for video xdmzFPxCY6w...
Fetching comments for video 8t0dDGhGL9Y...
Fetching comments for video edh3-Q1wfio...
Fetching comments for video idwhPzZkGBI...
Fetching comments for video xC_hIZt8xVU...
Fetching comments for video x7sX570aGi8...
Fetching comments for video 3ZCw7Cxkfzo...
Fetching comments for video qCNdjzK_pL8...
Fetching comments for video 5U2NrvCZ3u4...
Fetching comments for video 67R1gGF0hfk...
Fetching comments for video _dsuJXDi-iI...
Fetching comments for video AlMkP3vxxUY...
Fetching comments for video Jb5fyREGr-g...
Fetching comments for video nPxRmERyvHE...
Fetching comments for video ZSxe86t-ZeI...
Fetching comments for video 99mcDZIFiP4...
Fetching comments for video Uf6UjaMSG5Q...
Fetching comments for video n0g4A2-gp08...
Fetching comments for video Td4HEuw95zw...
Fetching comments for 

## Playlist Details

In [2]:
import requests
import csv

# API details
API_KEY = "YOUR_API_KEY"
CHANNEL_ID = "UCSaJZiQye_U2ZQXo85bQFLw"
BASE_URL_PLAYLISTS = "https://www.googleapis.com/youtube/v3/playlists"
BASE_URL_PLAYLIST_ITEMS = "https://www.googleapis.com/youtube/v3/playlistItems"
BASE_URL_VIDEOS = "https://www.googleapis.com/youtube/v3/videos"
PART_PLAYLISTS = "contentDetails,id,localizations,player,snippet,status"
PART_PLAYLIST_ITEMS = "contentDetails"
PART_VIDEOS = "statistics"
MAX_RESULTS = 50

# Function to get playlists data
def get_playlists_data(api_key, channel_id, max_results=50, page_token=None):
    params = {
        "key": api_key,
        "channelId": channel_id,
        "part": PART_PLAYLISTS,
        "maxResults": max_results,
    }
    if page_token:
        params["pageToken"] = page_token

    response = requests.get(BASE_URL_PLAYLISTS, params=params)
    response.raise_for_status()
    return response.json()

# Function to get video IDs from a playlist
def get_video_ids_from_playlist(api_key, playlist_id, max_results=50, page_token=None):
    video_ids = []
    while True:
        params = {
            "key": api_key,
            "playlistId": playlist_id,
            "part": PART_PLAYLIST_ITEMS,
            "maxResults": max_results,
        }
        if page_token:
            params["pageToken"] = page_token

        response = requests.get(BASE_URL_PLAYLIST_ITEMS, params=params)
        response.raise_for_status()
        data = response.json()

        video_ids.extend([item["contentDetails"]["videoId"] for item in data.get("items", [])])
        page_token = data.get("nextPageToken")
        if not page_token:
            break

    return video_ids

# Function to get view and like counts for a list of video IDs
def get_video_stats(api_key, video_ids):
    total_views = 0
    total_likes = 0
    for i in range(0, len(video_ids), 50):  # API supports up to 50 video IDs per call
        params = {
            "key": api_key,
            "id": ",".join(video_ids[i:i + 50]),
            "part": PART_VIDEOS,
        }
        response = requests.get(BASE_URL_VIDEOS, params=params)
        response.raise_for_status()
        data = response.json()

        total_views += sum(int(item["statistics"].get("viewCount", 0)) for item in data.get("items", []))
        total_likes += sum(int(item["statistics"].get("likeCount", 0)) for item in data.get("items", []) if "likeCount" in item["statistics"])
    return total_views, total_likes

# Function to extract relevant data from playlist response
def extract_playlist_info(api_key, playlists_data):
    playlists_info = []

    for item in playlists_data.get("items", []):
        snippet = item.get("snippet", {})
        content_details = item.get("contentDetails", {})
        player = item.get("player", {})
        status = item.get("status", {})
        localizations = item.get("localizations", {})

        playlist_id = item.get("id", "N/A")
        video_ids = get_video_ids_from_playlist(api_key, playlist_id)
        total_views, total_likes = get_video_stats(api_key, video_ids)

        playlists_info.append({
            "playlistId": playlist_id,
            "title": snippet.get("title", "N/A"),
            "description": snippet.get("description", "N/A"),
            "publishedAt": snippet.get("publishedAt", "N/A"),
            "channelId": snippet.get("channelId", "N/A"),
            "playlistItemsCount": content_details.get("itemCount", "N/A"),
            "embedHtml": player.get("embedHtml", "N/A"),
            "status": status.get("privacyStatus", "N/A"),
            "localizations": ", ".join([f"{key}: {value.get('title', 'N/A')}" for key, value in localizations.items()]),
            "totalViews": total_views,  # Add total views
            "totalLikes": total_likes   # Add total likes
        })

    return playlists_info

# Function to export data to CSV
def export_to_csv(data, filename="youtube_playlists_details.csv"):
    with open(filename, "w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        # Write headers
        writer.writerow([
            "Playlist ID", "Title", "Description", "Published At", "Channel ID",
            "Playlist Items Count", "Embed HTML", "Privacy Status", "Localizations", "Total Views", "Total Likes"
        ])
        # Write data rows
        for item in data:
            writer.writerow([
                item["playlistId"], item["title"], item["description"], item["publishedAt"],
                item["channelId"], item["playlistItemsCount"], item["embedHtml"],
                item["status"], item["localizations"], item["totalViews"], item["totalLikes"]
            ])

# Main function
if __name__ == "__main__":
    # Get playlists data from API
    next_page_token = None
    all_playlists_info = []

    while True:
        playlists_data = get_playlists_data(API_KEY, CHANNEL_ID, MAX_RESULTS, next_page_token)
        playlists_info = extract_playlist_info(API_KEY, playlists_data)
        all_playlists_info.extend(playlists_info)

        # Check if there are more pages to fetch
        next_page_token = playlists_data.get("nextPageToken")    
        if not next_page_token:
            break

    # Export the data to CSV
    export_to_csv(all_playlists_info)
    print(f"Exported playlist details to youtube_playlists_details.csv.")


Exported playlist details to youtube_playlists_details.csv.
